# Climate portfolio post

Your turn!

## Pick a new location and/or measurement to plot
🌏 📈

Recreate the workflow you just did in a place that interests you OR with
a different measurement. You will need to make your own new Markdown and
Code cells below this one, or create a new notebook.

Your analysis should include:

1.  A researched (with citations or links) **site description**,
    including *why* you chose the site
2.  A researched (with citations or links) **data description**,
    including a **data citation**
3.  A researched (with citations or links) **methods overview**
4.  Some kind of **visual evidence** (plot, chart, diagram) for your
    results
5.  A **headline and description** for the visual evidence that
    *interprets* your analysis and puts it *in context*

You should also delete the instructions before submitting a portfolio page.

To submit your portfolio post, we're going to do something a little different this week, to see if it is easier:

 1. Put it in your portfolio repository, along with any images or interactive html visualization
 2. Add a markdown file to the `posts/02-climate` folder that contains a link to your portfolio.

## Annual Temperatures in Seattle Wasthington

* Below is code to pull, plot, and fit observed temperatures for Rosalia Washington. 
* The city was chosen because it has a long history of temperature observations from 1901 onward. 
* Data citation https://www.ncdc.noaa.gov/cdo-web/datasets/GHCND/stations/GHCND:USC00457180/detail
* Methods overview

In [1]:
# Import required packages
import pandas as pd
# Advanced options on matplotlib/seaborn/pandas plots
import matplotlib.pyplot as plt
# Common statistical plots for tabular data
import seaborn as sns
# Fit an OLS linear regression
from sklearn.linear_model import LinearRegression

In [10]:
# import the data
# Create the URL
Washington_URL = ('https://www.ncei.noaa.gov/access/services/da'
'ta/v1?dataset=daily-summaries&dataTypes=TOBS&stations='
'USC00457180&startDate=2001-01-01&endDate=2023-09-08')
Washington_URL

'https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&dataTypes=TOBS&stations=USC00457180&startDate=2001-01-01&endDate=2023-09-08'

In [12]:
climate_df = pd.read_csv(
    Washington_URL,
    na_values=['NaN'])

climate_df

,STATION,DATE,TOBS
0,USC00457180,2001-01-01,-6.0
1,USC00457180,2001-01-02,-22.0
2,USC00457180,2001-01-03,-33.0
3,USC00457180,2001-01-04,22.0
4,USC00457180,2001-01-05,44.0
...,...,...,...
8211,USC00457180,2023-09-04,NaN
8212,USC00457180,2023-09-05,94.0
8213,USC00457180,2023-09-06,122.0
8214,USC00457180,2023-09-07,117.0
